## Introduction





## Investigation of Data Source

This database is downloaded from https://www.kaggle.com/anth7310/mental-health-in-the-tech-industry. In the description of this Kaggle database, we're told the following:
- The data is from Open Source Mental Illness (OMSI) using survey data from 2014 to 2019.
- The data has been modified by grouping similar questions together.
- Values in the answers table were changed to be more consistent (i.e. the value of 1 may have also appeared as 1.0) These have been formalised to be integers.
- Spelling errors were fixed.

Before we begin exploring the data, I took a look at the website where the data was collected https://osmihelp.org/research. Here it lists one survey conducted each year, ranging from 2014 to 2020, as well as the option to partake in the 2021 survey. All these surveys are covering the same topic, 'Attitudes Towards Mental Health in the Tech Workplace'. 

The Kaggle Database we're investigating was uploaded in September 2020, which means the results for the 2020 survey hadn't been released yet. Now that they have and are available to download, I plan to also add these to our analysis in the same way the other surveys have been. To do this I'll download an earlier dataset and compare it to how it's transformed into it's form in the Kaggle datasets.

The first and largest of the surveys (2014) has 2 video links that used this data:
- Mental Health in the Tech Industry | Ed Finkler | Talks at Google: https://www.youtube.com/watch?v=NHulgcO_16U&list=PL1MEC8mwrpaIdzYKRidvNB5eYSwWrqFZ3
- Supporting Mental Health in the Workplace: Creating a Culture of Support: https://www.youtube.com/watch?v=-RCScR9xDQw&list=PL1MEC8mwrpaKCW8piX6mntf6-620Efncf

I watched through both of these videos to get a better perspective of the topic surrounding the data before digging into it.

Another crucial finding is that this site also has a 'Basic Report' for every completed survey. these basic reports show every question and the percentages of answers selected. Take a look at the 2020 Basic Report here: https://osmi.typeform.com/report/A7mlxC/itVHRYbNRnPqDI9C.

Due to these reports being available, I'm not going to focus much on the distributions of answers and focus on specific trends we see instead.



## The Kaggle Dataset

In [1]:
import sqlite3 as sql
import pandas as pd

The database **mental_health** contains the following datasets:
- Answer - (PRIMARY/FOREIGN KEY SurveyID, PRIMARY KEY UserID, PRIMARY/FOREIGN KEY QuestionID, TEXT AnswerText)
- Question - (PRIMARY KEY QuestionID, TEXT QuestionText)
- Survey - (PRIMARY KEY INT SurveyID, TEXT Description)

In [2]:
conn = sql.connect('data/mental_health.sqlite')

In [3]:
answer = pd.read_sql('SELECT * FROM Answer', conn)
question = pd.read_sql('SELECT * FROM Question', conn)
survey = pd.read_sql('SELECT * FROM Survey', conn)